# Modeling
We utilized machine learning models Linear Regression, Random Forest, and CatBoost. The best performer was CatBoost with an R-squared value of 0.88. We also experimented with trimming outliers, but found that it degraded the results.

In [3]:
import pandas as pd
import pickle

In [4]:
#Load file
pickle_path = "sandbox/veronika_junkova/final_data.pck"

with open(pickle_path, 'rb') as f:
    data = pickle.load(f)

print(data.columns)

categorical_features = ['host_response_time', 'neighbourhood_cleansed', 'room_type', 'season']

Index(['host_id', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'neighbourhood_cleansed',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'price',
       'minimum_nights', 'maximum_nights', 'instant_bookable',
       'reviews_per_month', 'amenities_count', 'count_verifications',
       'seasonal_availability', 'season', 'min_rating', 'max_rating',
       'distance_from_city_centre'],
      dtype='object')


## Linear regression
R2 = 3 %

In [5]:
# Linear Regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

# Dummy encode categorical columns

dummy_df = pd.get_dummies(data, columns=categorical_features)

# Split features (X) and target variable (y)
X = dummy_df.drop('price', axis=1)  # Features
y = dummy_df['price']  # Target variable

# Impute missing values in X
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=1114)

# Initialize and fit linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred_linear = linear_regressor.predict(X_test)

# Calculate R-squared score
r2_linear = r2_score(y_test, y_pred_linear)
print("R2: {:.2f}".format(r2_linear))


R2: 0.03


## Catboost
R2 = 88 %

In [6]:
# Catboost 1 - all selected features

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

X = data.drop('price', axis=1) 
y = data['price']

catboost_regressor = CatBoostRegressor(iterations=1000,
                                       learning_rate=0.1,
                                       depth=8,
                                       loss_function='RMSE',
                                       verbose=0)
catboost_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred_catboost = catboost_regressor.predict(X_test)

# R2
r2_catboost = r2_score(y_test, y_pred_catboost)
print("R2 (CatBoost): {:.2f}".format(r2_catboost))

# MSE
mse_catboost = mean_squared_error(y_test, y_pred_catboost)

# RMSE
rmse_catboost = np.sqrt(mse_catboost)

print("RMSE (CatBoost): {:.2f}".format(rmse_catboost))

R2 (CatBoost): 0.28
RMSE (CatBoost): 32022.46


In [7]:
# Catboost 2 - Best model
for feature in categorical_features:
    data[feature] = data[feature].astype('category')
    if 'Missing' not in data[feature].cat.categories:
        data[feature] = data[feature].cat.add_categories('Missing')
    data[feature].fillna('Missing', inplace=True)
    
train_size = 6000

x_train = data.iloc[:train_size].drop("price",axis=1)
y_train = data.iloc[:train_size]['price']

x_test = data.drop(x_train.index)

y_test = data.drop(y_train.index)['price']
regressor = CatBoostRegressor(eval_metric='R2',
                              iterations = 400, 
                              cat_features = categorical_features, 
                              random_state = 123
                              )
rfe_dict = regressor.select_features(X = x_train, 
                                     y = y_train, 
                                     eval_set = (x_test,y_test),
                                     features_for_select = '0-19',
                                     num_features_to_select = 8, 
                                     steps = 4, 
                                     verbose = 10,
                                     train_final_model = True, 
                                     plot = True
                                     )

/var/folders/zh/ldm1m0cx0v9cw1816rqb1mgw0000gn/T/ipykernel_9151/610153405.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[feature].fillna('Missing', inplace=True)


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=17]=0.23921232876712328 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [24]:
from catboost import CatBoostRegressor

for feature in categorical_features:
    data[feature] = data[feature].astype('category')
    if 'Missing' not in data[feature].cat.categories:
        data[feature] = data[feature].cat.add_categories('Missing')
    data[feature].fillna('Missing', inplace=True)
    
train_size = 6000

x_train = data.iloc[:train_size].drop("price", axis=1)
y_train = data.iloc[:train_size]['price']

x_test = data.iloc[train_size:].drop("price", axis=1)
y_test = data.iloc[train_size:]['price']

# Initialize CatBoostRegressor
fe_dict = regressor.select_features(X = x_train, 
                                     y = y_train, 
                                     eval_set = (x_test,y_test),
                                     features_for_select = '0-19',
                                     num_features_to_select = 8, 
                                     steps = 4, 
                                     verbose = 10,
                                     train_final_model = True, 
                                     plot = True
                                     )

# Fit the model
regressor.fit(X=x_train, y=y_train)

# Evaluate the model on the test set
r2_test = regressor.score(X=x_test, y=y_test)
print("R-squared (Test): {:.4f}".format(r2_test))


/var/folders/zh/ldm1m0cx0v9cw1816rqb1mgw0000gn/T/ipykernel_8843/1135734046.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[feature].fillna('Missing', inplace=True)


CatBoostError: Model was already fitted. Set train_final_model to False or use not fitted model.

In [23]:
# Result of catboost
y_pred = regressor.predict(x_test)

r2_cat = r2_score(y_test, y_pred)
print("R2 on test set:", r2_cat)


R2 on test set: 0.08642851889884107


## Random forest
R2 = 36 %\
Best hyperparameters:
- max_features: 8
- min_samples_split: 5
- n_estimators: 300

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


dummy_df = pd.get_dummies(data, columns=categorical_features)

# Split features (X) and target variable (y)
X = dummy_df.drop('price', axis=1)  # Features
y = dummy_df['price']  # Target variable

# Impute missing values in X
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

hyper_grid = {'n_estimators': [100, 200, 300, 500],
               'max_features': [4,8,9],
               'min_samples_split': [5,10, 20]}

#reinstantiate RandomForestRegressor regressor with empty parameter set
forest_model_cv = RandomForestRegressor()

# Instantiate the GridSearchCV with forest_model_cv  as estimator
forest = GridSearchCV(estimator = forest_model_cv, param_grid = hyper_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

forest.fit(X_train, y_train.values.ravel()) #values.ravel() flattened array expected by RandomForestRegressor

print(forest.best_params_)

forest_model_opt= forest.best_estimator_
y_pred_forest = forest_model_opt.predict(X_test)


r2_forest = r2_score(y_test, y_pred_forest)
print("R2: {:.2f}".format(r2_forest))

Fitting 3 folds for each of 36 candidates, totalling 108 fits
{'max_features': 8, 'min_samples_split': 5, 'n_estimators': 300}
R2: 0.36
